# Intro to AISQL - Natural Language Movie Insights

## Overview

Welcome to an introduction to **AISQL** (Artificial Intelligence SQL), Snowflake's powerful feature that allows you to query your data using natural language instead of complex SQL syntax. In this template, you'll learn how AISQL can transform the way you interact with data by enabling conversational queries.

### What You'll Learn
- How to use AISQL for natural language data queries
- Practical examples using movie and ratings data
- Benefits of AISQL for business users and analysts
- Best practices for writing effective natural language queries

### Business Context
Movie streaming services and entertainment companies analyze vast amounts of data to understand viewer preferences, identify trending content, and make data-driven decisions. Traditional SQL analysis requires technical expertise, but AISQL democratizes data analysis by allowing business users to ask questions in plain English.

**Estimated Time:** Under 5 minutes

## Step 1: Environment Setup

First, let's set up our Snowflake environment using the standard learning environment configuration.

In [ ]:
-- Set up the standard learning environment
USE ROLE SNOWFLAKE_LEARNING_ROLE;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
USE DATABASE SNOWFLAKE_LEARNING_DB;

-- Create a unique schema for this template
SET schema_name = CONCAT(CURRENT_USER(), '_AISQL');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($schema_name);
USE SCHEMA IDENTIFIER($schema_name);

SELECT 'Environment ready! Using schema: ' || CURRENT_SCHEMA() as setup_status;

## Step 2: Data Preparation

Let's clean up any existing objects and create our sample movie dataset with movies and their ratings.

In [ ]:
-- Clean up any existing objects from previous runs
DROP TABLE IF EXISTS MOVIES;
DROP TABLE IF EXISTS MOVIE_RATINGS;
DROP TABLE IF EXISTS MOVIE_REVIEWS;
DROP VIEW IF EXISTS MOVIE_ANALYSIS_VIEW;

SELECT 'Cleanup completed successfully' as cleanup_status;

In [ ]:
-- Create movies table with sample data
CREATE OR REPLACE TABLE MOVIES (
    movie_id INT,
    title STRING,
    year INT,
    genre STRING,
    director STRING,
    duration_minutes INT,
    budget_millions FLOAT,
    box_office_millions FLOAT
);

-- Insert sample movie data
INSERT INTO MOVIES VALUES
    (1, 'The Shawshank Redemption', 1994, 'Drama', 'Frank Darabont', 142, 25, 16),
    (2, 'The Godfather', 1972, 'Crime', 'Francis Ford Coppola', 175, 6, 135),
    (3, 'The Dark Knight', 2008, 'Action', 'Christopher Nolan', 152, 185, 1005),
    (4, 'Pulp Fiction', 1994, 'Crime', 'Quentin Tarantino', 154, 8, 214),
    (5, 'Forrest Gump', 1994, 'Drama', 'Robert Zemeckis', 142, 55, 678),
    (6, 'Inception', 2010, 'Sci-Fi', 'Christopher Nolan', 148, 160, 837),
    (7, 'The Matrix', 1999, 'Sci-Fi', 'The Wachowskis', 136, 63, 465),
    (8, 'Goodfellas', 1990, 'Crime', 'Martin Scorsese', 146, 25, 47),
    (9, 'Interstellar', 2014, 'Sci-Fi', 'Christopher Nolan', 169, 165, 677),
    (10, 'Parasite', 2019, 'Thriller', 'Bong Joon-ho', 132, 11, 258);

SELECT COUNT(*) as movies_created FROM MOVIES;

In [ ]:
-- Create ratings table with sample data
CREATE OR REPLACE TABLE MOVIE_RATINGS (
    movie_id INT,
    imdb_rating FLOAT,
    rotten_tomatoes_score INT,
    metacritic_score INT,
    num_reviews INT
);

-- Insert rating data
INSERT INTO MOVIE_RATINGS VALUES
    (1, 9.3, 91, 80, 2500000),
    (2, 9.2, 97, 100, 1800000),
    (3, 9.0, 94, 84, 2800000),
    (4, 8.9, 92, 94, 2000000),
    (5, 8.8, 71, 82, 2100000),
    (6, 8.8, 87, 74, 2300000),
    (7, 8.7, 88, 73, 1900000),
    (8, 8.7, 96, 90, 1200000),
    (9, 8.6, 72, 74, 1800000),
    (10, 8.5, 99, 96, 700000);

SELECT COUNT(*) as ratings_created FROM MOVIE_RATINGS;

## Step 3: Exploring Our Dataset

Let's take a quick look at our movie data to understand what we're working with.

In [ ]:
-- Explore our movie dataset
SELECT 
    'Movies table' as table_name,
    COUNT(*) as row_count 
FROM MOVIES

UNION ALL

SELECT 
    'Ratings table' as table_name,
    COUNT(*) as row_count 
FROM MOVIE_RATINGS;

-- Sample movie data
SELECT * FROM MOVIES LIMIT 3;

## Step 4: Introduction to AISQL

**AISQL** (Artificial Intelligence SQL) is Snowflake's revolutionary feature that allows you to query your data using natural language instead of complex SQL syntax. This democratizes data analysis by making it accessible to business users who may not be SQL experts.

### Key Benefits:
- **Natural Language Queries** - Ask questions in plain English
- **Faster Insights** - Get answers conversationally without writing SQL
- **Accessible to Everyone** - No technical SQL knowledge required
- **Accurate Results** - AI understands context and generates proper SQL

Let's see AISQL in action with some practical examples!

### Example 1: Finding Top-Rated Movies

**Natural Language Question:** "Show me the top 5 highest-rated movies on IMDB"

With AISQL, you could simply ask this question in natural language. For comparison, here's the traditional SQL approach:

In [ ]:
-- Traditional SQL: Top 5 highest-rated movies
SELECT 
    m.title, 
    m.year, 
    m.genre, 
    r.imdb_rating
FROM MOVIES m
JOIN MOVIE_RATINGS r ON m.movie_id = r.movie_id
ORDER BY r.imdb_rating DESC
LIMIT 5;

💡 **With AISQL**, you could simply ask: *"Show me the top 5 movies by IMDB rating"* and get the same results!

### Example 2: Genre Analysis

**Natural Language Question:** "What genres have the highest average ratings?"

In [ ]:
-- Genre popularity analysis
SELECT 
    m.genre, 
    AVG(r.imdb_rating) as avg_imdb_rating,
    AVG(r.rotten_tomatoes_score) as avg_rt_score,
    COUNT(*) as movie_count
FROM MOVIES m
JOIN MOVIE_RATINGS r ON m.movie_id = r.movie_id
GROUP BY m.genre
ORDER BY avg_imdb_rating DESC;

### Example 3: Director Performance

**Natural Language Question:** "Which directors have made the most successful movies?"

In [ ]:
-- Director performance analysis
SELECT 
    m.director,
    COUNT(*) as movie_count,
    AVG(r.imdb_rating) as avg_rating,
    AVG(m.box_office_millions) as avg_box_office
FROM MOVIES m
JOIN MOVIE_RATINGS r ON m.movie_id = r.movie_id
GROUP BY m.director
HAVING COUNT(*) >= 1
ORDER BY avg_rating DESC, avg_box_office DESC;

### Example 4: Financial Performance

**Natural Language Question:** "Which movies had the best return on investment?"

In [ ]:
-- Return on investment analysis
SELECT 
    m.title,
    m.budget_millions,
    m.box_office_millions,
    ROUND((m.box_office_millions / m.budget_millions), 2) as roi_ratio,
    r.imdb_rating
FROM MOVIES m
JOIN MOVIE_RATINGS r ON m.movie_id = r.movie_id
WHERE m.budget_millions > 0
ORDER BY roi_ratio DESC
LIMIT 5;

## Step 5: AISQL Best Practices

Now that you've seen AISQL in action, here are some best practices for writing effective natural language queries:

### 🎯 **Best Practices**
- **Be Specific**: Use clear, descriptive language in your questions
- **Use Context**: Reference table and column names when helpful
- **Start Simple**: Begin with basic questions and build complexity gradually
- **Be Conversational**: Ask questions as you would to a human analyst

### 💡 **Example Natural Language Questions for AISQL**
- "Show me the top 10 movies by IMDB rating"
- "What's the average box office for sci-fi movies?"
- "Which director has the highest average movie rating?"
- "Find movies from 2010 to 2020 with ratings above 8.5"
- "What's the total box office for Christopher Nolan movies?"

### 🚀 **When to Use AISQL vs Traditional SQL**
- **Use AISQL for**: Quick exploratory analysis, ad-hoc questions, business user queries
- **Use Traditional SQL for**: Complex transformations, production pipelines, performance-critical queries

## Key Takeaways

🎉 **Congratulations!** You've successfully learned the fundamentals of AISQL. Here's what you've accomplished:

✅ **Understanding AISQL**: Learned how natural language queries work in Snowflake

✅ **Practical Examples**: Saw real-world movie data analysis using both traditional SQL and AISQL approaches

✅ **Business Applications**: Explored how AISQL democratizes data analysis for business users

✅ **Best Practices**: Discovered tips for writing effective natural language queries

### 🔑 **Key Benefits of AISQL**
- **Democratizes Data Access** - Anyone can query data without SQL expertise
- **Faster Time to Insights** - Get answers through conversational queries
- **Reduces Technical Barriers** - Business users can self-serve data needs
- **Maintains Accuracy** - AI generates proper SQL behind the scenes

## Step 6: Cleanup

Let's clean up the resources we created during this template.

In [ ]:
-- Clean up all created resources
DROP TABLE IF EXISTS MOVIES;
DROP TABLE IF EXISTS MOVIE_RATINGS;
DROP TABLE IF EXISTS MOVIE_REVIEWS;

SELECT 'All resources cleaned up successfully!' as cleanup_status;

## Additional Resources

Ready to explore more? Here are some helpful resources:

### 📖 **Documentation**
- [Snowflake AISQL Documentation](https://docs.snowflake.com/user-guide/snowflake-cortex/aisql) - Complete guide to AISQL features
- [Snowflake Cortex](https://docs.snowflake.com/user-guide/snowflake-cortex) - AI and ML capabilities in Snowflake

### 🎯 **Related Templates**
- Explore more templates at [Snowflake Templates](https://app.snowflake.com/templates)
- Try advanced analytics and AI/ML templates

### 🚀 **Next Steps**
1. **Try AISQL with your own data** - Apply these concepts to your real datasets
2. **Explore advanced features** - Learn about complex AISQL capabilities
3. **Share with your team** - Introduce AISQL to business users in your organization

---

**Thank you for completing this AISQL introduction!** 🎬✨

You're now equipped to leverage the power of natural language data analysis with Snowflake AISQL.